In [2]:
pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.2/571.2 kB 289.6 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 335.2 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from elasticsearch import Elasticsearch

# # Create an Elasticsearch client instance with authentication
# es = Elasticsearch(
#     "https://localhost:9200",  # Use https for SSL
#     basic_auth=(
#         "elastic",
#         "8+6x+mXMgyoyi0F7Fi07",
#     ),  # Replace with your username and password
#     verify_certs=False,  # Optional: Disable SSL certificate verification for development
#     ssl_show_warn=False,  # This will disable the warning
# )
es = Elasticsearch(
    "http://84.16.230.94:9200", basic_auth=("elastic", "c*+Nbwk82gA7RQfFHkQ6")
)

create a new index with documents for testing


In [4]:
# Create the department index with mappings
create_index_body = {
    "mappings": {
        "properties": {
            "name": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
            # "employees": {"type": "nested"},
        }
    }
}
es.indices.delete(index="department", ignore=[400, 404])
# Create the index
es.indices.create(
    index="department", body=create_index_body, ignore=400
)  # ignore=400 to ignore if already exists

# Define the test documents
department_1 = {
    "name": "Development",
    "employees": [
        {
            "name": "Eric Green",
            "age": 39,
            "gender": "M",
            "position": "Big Data Specialist",
        },
        {
            "name": "James Taylor",
            "age": 27,
            "gender": "M",
            "position": "Software Developer",
        },
        {"name": "Gary Jenkins", "age": 21, "gender": "M", "position": "Intern"},
        {"name": "Julie Powell", "age": 26, "gender": "F", "position": "Intern"},
        {
            "name": "Benjamin Smith",
            "age": 46,
            "gender": "M",
            "position": "Senior Software Engineer",
        },
    ],
}

department_2 = {
    "name": "HR & Marketing",
    "employees": [
        {
            "name": "Patricia Lewis",
            "age": 42,
            "gender": "F",
            "position": "Senior Marketing Manager",
        },
        {"name": "Maria Anderson", "age": 56, "gender": "F", "position": "Head of HR"},
        {"name": "Margaret Harris", "age": 19, "gender": "F", "position": "Intern"},
        {
            "name": "Ryan Nelson",
            "age": 31,
            "gender": "M",
            "position": "Marketing Manager",
        },
        {
            "name": "Kathy Williams",
            "age": 49,
            "gender": "F",
            "position": "Senior Marketing Manager",
        },
        {
            "name": "Jacqueline Hill",
            "age": 28,
            "gender": "F",
            "position": "Junior Marketing Manager",
        },
        {
            "name": "Donald Morris",
            "age": 39,
            "gender": "M",
            "position": "SEO Specialist",
        },
        {"name": "Evelyn Henderson", "age": 24, "gender": "F", "position": "Intern"},
        {
            "name": "Earl Moore",
            "age": 21,
            "gender": "M",
            "position": "Junior SEO Specialist",
        },
        {
            "name": "Phillip Sanchez",
            "age": 35,
            "gender": "M",
            "position": "SEM Specialist",
        },
    ],
}

# Add the documents to the index
# es.index(index="department", id=1, body=department_1)
# es.index(index="department", id=2, body=department_2)

# print("Department index created and documents added.")

/tmp/ipykernel_10463/1756142033.py:10: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="department", ignore=[400, 404])


AuthenticationException: AuthenticationException(401, 'security_exception', 'unable to authenticate user [elastic] for REST request [/department]')

In [ ]:
# Define the mapping update to add the join_field
update_mapping_body = {
    "properties": {
        "join_field": {"type": "join", "relations": {"department": "employee"}}
    }
}
# Update the department index mapping
response = es.indices.put_mapping(index="department", body=update_mapping_body)

print("Mapping updated:", response)

adding departments and employees


In [ ]:
# Adding departments
es.index(
    index="department", id=1, body={"name": "Development", "join_field": "department"}
)

es.index(
    index="department", id=2, body={"name": "Marketing", "join_field": "department"}
)

# Adding employees for departments
es.index(
    index="department",
    id=3,
    routing=1,
    body={
        "name": "Bo Andersen",
        "age": 28,
        "gender": "M",
        "join_field": {"name": "employee", "parent": 1},
    },
)

es.index(
    index="department",
    id=4,
    routing=2,
    body={
        "name": "John Doe",
        "age": 44,
        "gender": "M",
        "join_field": {"name": "employee", "parent": 2},
    },
)

es.index(
    index="department",
    id=5,
    routing=1,
    body={
        "name": "James Evans",
        "age": 32,
        "gender": "M",
        "join_field": {"name": "employee", "parent": 1},
    },
)

es.index(
    index="department",
    id=6,
    routing=1,
    body={
        "name": "Daniel Harris",
        "age": 52,
        "gender": "M",
        "join_field": {"name": "employee", "parent": 1},
    },
)

es.index(
    index="department",
    id=7,
    routing=2,
    body={
        "name": "Jane Park",
        "age": 23,
        "gender": "F",
        "join_field": {"name": "employee", "parent": 2},
    },
)

es.index(
    index="department",
    id=8,
    routing=1,
    body={
        "name": "Christina Parker",
        "age": 29,
        "gender": "F",
        "join_field": {"name": "employee", "parent": 1},
    },
)

print("Departments and employees added successfully.")

getting childs of a certain parent by id


In [ ]:
# Perform search to find employees with a parent_id of 1
response = es.search(
    index="department", body={"query": {"parent_id": {"type": "employee", "id": 1}}}
)

# Print the search results
for hit in response["hits"]["hits"]:
    print(hit["_source"])

get children based on parent's name


In [ ]:
# Perform search for employees in the "Development" department
response = es.search(
    index="department",
    body={
        "query": {
            "has_parent": {
                "parent_type": "department",
                "query": {"term": {"name.keyword": "Development"}},
            }
        }
    },
)

# Print search results
for hit in response["hits"]["hits"]:
    print(hit["_source"])

print("\n\n\n")

# Perform search with parent score relevance
response_with_score = es.search(
    index="department",
    body={
        "query": {
            "has_parent": {
                "parent_type": "department",
                "score": True,
                "query": {"term": {"name.keyword": "Development"}},
            }
        }
    },
)

# Print search results with relevance scores
for hit in response_with_score["hits"]["hits"]:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

In [ ]:
# Search for departments with employees who match the specified bool query
response = es.search(
    index="department",
    body={
        "query": {
            "has_child": {
                "type": "employee",
                "query": {
                    "bool": {
                        "must": [{"range": {"age": {"gte": 50}}}],
                        "should": [{"term": {"gender.keyword": "M"}}],
                    }
                },
            }
        }
    },
)

# Print search results
for hit in response["hits"]["hits"]:
    print(hit["_source"])


print("\n\n\n")


# Search for departments with employees, scoring relevance using score_mode
response_with_score = es.search(
    index="department",
    body={
        "query": {
            "has_child": {
                "type": "employee",
                "score_mode": "sum",
                "query": {
                    "bool": {
                        "must": [{"range": {"age": {"gte": 50}}}],
                        "should": [{"term": {"gender.keyword": "M"}}],
                    }
                },
            }
        }
    },
)

# Print search results with relevance scores
for hit in response_with_score["hits"]["hits"]:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")


print("\n\n\n")

# Search for departments with between 2 and 5 employees matching the query
response_min_max = es.search(
    index="department",
    body={
        "query": {
            "has_child": {
                "type": "employee",
                "score_mode": "sum",
                "min_children": 1,
                "max_children": 5,
                "query": {
                    "bool": {
                        "must": [{"range": {"age": {"gte": 50}}}],
                        "should": [{"term": {"gender.keyword": "M"}}],
                    }
                },
            }
        }
    },
)

# Print search results with relevance scores and child count constraints
for hit in response_min_max["hits"]["hits"]:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

# Multi level relations

employe -> department -> company <- supplier


create the mapping and add some data


In [ ]:
# Create the index with the specified mappings
es.options(ignore_status=[400, 404]).indices.create(
    index="company",
    body={
        "mappings": {
            "properties": {
                "join_field": {
                    "type": "join",
                    "relations": {
                        "company": ["department", "supplier"],
                        "department": "employee",
                    },
                }
            }
        }
    },
)

# add company
es.index(
    index="company", id=1, body={"name": "My Company Inc.", "join_field": "company"}
)

# add department to the company added above
es.index(
    index="company",
    id=2,
    routing=1,
    body={"name": "Development", "join_field": {"name": "department", "parent": 1}},
)

# add employee to the department
es.index(
    index="company",
    id=3,
    routing=1,
    body={"name": "Bo Andersen", "join_field": {"name": "employee", "parent": 2}},
)

# Add another company
es.index(
    index="company",
    id=4,
    body={"name": "Another Company, Inc.", "join_field": "company"},
)

# Add a department under "Another Company, Inc."
es.index(
    index="company",
    id=5,
    routing=4,
    body={"name": "Marketing", "join_field": {"name": "department", "parent": 4}},
)

# Add an employee to the Marketing department
es.index(
    index="company",
    id=6,
    routing=4,
    body={"name": "John Doe", "join_field": {"name": "employee", "parent": 5}},
)

In [ ]:
# Search for companies with departments having an employee named "John Doe"
response = es.search(
    index="company",
    body={
        "query": {
            "has_child": {
                "type": "department",
                "query": {
                    "has_child": {
                        "type": "employee",
                        "query": {"term": {"name.keyword": "Bo Andersen"}},
                    }
                },
            }
        }
    },
)

# Print search results
for hit in response["hits"]["hits"]:
    print(hit["_source"])

return innerhits


In [ ]:
# Perform has_child query with inner_hits for employee child documents
response = es.search(
    index="department",
    body={
        "query": {
            "has_child": {
                "type": "employee",
                "inner_hits": {},  # Enables inner hits
                "query": {
                    "bool": {
                        "must": [{"range": {"age": {"gte": 50}}}],
                        "should": [{"term": {"gender.keyword": "M"}}],
                    }
                },
            }
        }
    },
)

# Print results with inner hits
for hit in response["hits"]["hits"]:
    print(f"Department: {hit['_source']}")
    if "inner_hits" in hit:
        for inner_hit in hit["inner_hits"]["employee"]["hits"]["hits"]:
            print(f"  Employee: {inner_hit['_source']}")

In [ ]:
# Perform has_parent query with inner_hits for department parent documents
response = es.search(
    index="department",
    body={
        "query": {
            "has_parent": {
                "parent_type": "department",
                "inner_hits": {},  # Enables inner hits
                "query": {"term": {"name.keyword": "Development"}},
            }
        }
    },
)

# Print results with inner hits
for hit in response["hits"]["hits"]:
    print(f"Employee: {hit['_source']}")
    if "inner_hits" in hit:
        for inner_hit in hit["inner_hits"]["department"]["hits"]["hits"]:
            print(f"  Department: {inner_hit['_source']}")

In [ ]:
# Define users and stories data
users_data = [
    {"id": 1, "body": {"name": "John Roberts", "following": [2, 3]}},
    {"id": 2, "body": {"name": "Elizabeth Ross", "following": []}},
    {"id": 3, "body": {"name": "Jeremy Brooks", "following": [1, 2]}},
    {"id": 4, "body": {"name": "Diana Moore", "following": [3, 1]}},
]

stories_data = [
    {"id": 1, "body": {"user": 3, "content": "Wow look, a penguin!"}},
    {
        "id": 2,
        "body": {"user": 1, "content": "Just another day at the office... #coffee"},
    },
    {
        "id": 3,
        "body": {
            "user": 1,
            "content": "Making search great again! #elasticsearch #elk",
        },
    },
    {
        "id": 4,
        "body": {
            "user": 4,
            "content": "Had a blast today! #rollercoaster #amusementpark",
        },
    },
    {
        "id": 5,
        "body": {
            "user": 4,
            "content": "Yay, I just got hired as an Elasticsearch consultant - so excited!",
        },
    },
    {
        "id": 6,
        "body": {
            "user": 2,
            "content": "Chilling at the beach @ Greece #vacation #goodtimes",
        },
    },
]

# Create and populate users index
es.indices.create(index="users", ignore=400)  # ignore 400 (Index Already Exists)
for user in users_data:
    es.index(index="users", id=user["id"], body=user["body"])

# Create and populate stories index
es.indices.create(index="stories", ignore=400)  # ignore 400 (Index Already Exists)
for story in stories_data:
    es.index(index="stories", id=story["id"], body=story["body"])

print("Data added successfully.")

the next command, for each user it gets the following of him and display their stories


In [9]:
# Perform the search query
query = {
    "query": {"terms": {"user": {"index": "users", "id": "1", "path": "following"}}}
}

# Execute the search on the stories index
response = es.search(index="stories", body=query)

# Print the results
print("Search results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [users]', users, index_or_alias)

limitations

parents and children must be in the same index

 1 join field per index, but no limits on the number of relations

 1 parent per children, many to many relations is not feasable

# Testing joins

In [ ]:
query = {
    "size": 1,  # Limit the result to 1 document
    "query": {
        "bool": {
            "must": [
                {"term": {"join_field": "employee"}},
                {"range": {"employee.salary": {"gt": 70000}}},
            ]
        }
    },
}

response = es.search(index="department", body=query)
print("One Employee with Salary > 70,000:", response)

In [ ]:
from elasticsearch import Elasticsearch

es.indices.delete(index="marianeee", ignore=404)
# Define the mapping with explicit fields for department and employee
create_index_body = {
    "mappings": {
        "properties": {
            # Fields for Department
            "department": {
                "properties": {
                    "name": {
                        "type": "text",
                        "fields": {"keyword": {"type": "keyword"}},
                    },
                    "location": {"type": "text"},
                    "budget": {"type": "float"},
                }
            },
            # Fields for Employee
            "employee": {
                "properties": {
                    "name": {
                        "type": "text",
                        "fields": {"keyword": {"type": "keyword"}},
                    },
                    "position": {"type": "text"},
                    "salary": {"type": "float"},
                    "date_of_joining": {"type": "date"},
                }
            },
        }
    }
}

# Create the index
index_name = "marianeee"
response = es.indices.create(index=index_name, body=create_index_body)
print(f"Index '{index_name}' created with response:", response)